In [1]:
import os
import math
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

# with the new api
from importnb import imports
with imports("ipynb"):
    from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## PM2.5 Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Total Tangara Sensors: 18


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.461380,-76.512222
1,TANGARA_2BDE,d29dbmw,D29ESP32DE52BDE,3.3267974853515625 -76.62071228027344,3.326797,-76.620712
2,TANGARA_39D6,d29dbmw,D29ESP32DE539D6,3.3267974853515625 -76.62071228027344,3.326797,-76.620712
3,TANGARA_3B7E,d29dbmw,D29ESP32DE53B7E,3.3267974853515625 -76.62071228027344,3.326797,-76.620712
4,TANGARA_3BEA,d29dbmw,D29ESP32DE53BEA,3.3267974853515625 -76.62071228027344,3.326797,-76.620712


In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME = os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME = os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

# GROUP BY TIME
GROUP_BY_TIME = os.getenv("GROUP_BY_TIME", None)

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')
print(f"Group by Time: {GROUP_BY_TIME}")

Since: 2023-11-17T00:00:00-05:00 -> 1700197200000, Until: 2023-11-18T23:59:59-05:00 -> 1700369999000
Group by Time: 30s


In [4]:
# PM2.5 Data Frame Sensors
df_pm25_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'pm25', GROUP_BY_TIME)
df_pm25_raw.head()
#df_pm25_raw.shape

,TANGARA_260A,TANGARA_2BDE,TANGARA_39D6,TANGARA_3B7E,TANGARA_3BEA,TANGARA_421A,TANGARA_5D62,TANGARA_5636,TANGARA_D282,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,,,,,,,,,
2023-11-17 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,8.0,NaN
2023-11-17 00:00:30-05:00,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,7.0,10.0,5.0,9.0,10.0,4.0,7.0,9.0
2023-11-17 00:01:00-05:00,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,9.0,10.0,7.0,8.0,10.0,8.0,7.0,9.0
2023-11-17 00:01:30-05:00,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,10.0,11.0,11.0,7.0,10.0,6.0,8.0,8.0
2023-11-17 00:02:00-05:00,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,9.0,11.0,10.0,8.0,10.0,5.0,7.0,8.0


## Descriptive Statistics

In [5]:
# Describe Data
df_pm25_raw.describe()

,TANGARA_260A,TANGARA_2BDE,TANGARA_39D6,TANGARA_3B7E,TANGARA_3BEA,TANGARA_421A,TANGARA_5D62,TANGARA_5636,TANGARA_D282,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
count,5727.00000,1359.000000,1.0,1292.000000,1376.000000,1425.000000,2175.000000,1304.000000,1435.000000,5726.000000,5665.000000,4978.000000,4810.000000,5718.000000,5711.000000,5733.000000,5739.000000,5715.000000
mean,8.64938,5.729213,19.0,4.528638,4.756541,4.750877,5.322299,6.046779,5.514286,11.699616,9.561342,8.166533,7.926403,7.883176,9.531956,7.238095,8.528664,9.628696
std,5.12304,3.547318,NaN,2.849156,3.045765,3.036131,4.110204,3.497411,3.319195,16.372879,5.945327,5.151530,5.814738,4.034657,13.733579,7.289115,7.589274,4.943717
min,1.00000,1.000000,19.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
25%,5.00000,3.000000,19.0,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,5.000000,5.000000,3.000000,5.000000,5.000000,3.000000,4.000000,6.000000
50%,8.00000,5.000000,19.0,4.000000,4.000000,4.000000,4.000000,5.000000,5.000000,8.000000,9.000000,8.000000,7.000000,8.000000,8.000000,6.000000,7.000000,9.000000
75%,11.00000,7.000000,19.0,6.000000,6.000000,6.000000,6.000000,8.000000,7.000000,15.000000,13.000000,10.000000,11.000000,10.000000,10.000000,9.000000,11.000000,12.000000
max,33.00000,18.000000,19.0,18.000000,18.000000,17.000000,63.000000,17.000000,18.000000,310.000000,57.000000,95.000000,37.000000,22.000000,339.000000,195.000000,264.000000,61.000000


## Missing Data

In [6]:
import warnings
warnings.filterwarnings('ignore')

# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_pm25_raw.columns:
    df_pm25_sensor = df_pm25_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_pm25_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%
Tangara Sensor: TANGARA_2BDE, Data: 24%, Missing: 76%, To be checked
Tangara Sensor: TANGARA_39D6, Data: 0%, Missing: 100%, To be checked
Tangara Sensor: TANGARA_3B7E, Data: 22%, Missing: 78%, To be checked
Tangara Sensor: TANGARA_3BEA, Data: 24%, Missing: 76%, To be checked
Tangara Sensor: TANGARA_421A, Data: 25%, Missing: 75%, To be checked
Tangara Sensor: TANGARA_5D62, Data: 38%, Missing: 62%, To be checked
Tangara Sensor: TANGARA_5636, Data: 23%, Missing: 77%, To be checked
Tangara Sensor: TANGARA_D282, Data: 25%, Missing: 75%, To be checked
Tangara Sensor: TANGARA_2B42, Data: 86%, Missing: 14%, To be checked
Tangara Sensor: TANGARA_2E9A, Data: 84%, Missing: 16%, To be checked


## Data Correlation

In [7]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_pm25_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_pm25_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
if not math.isnan(df_reference_sensor.std()):
    is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
    if not is_ok:
        print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_pm25_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_pm25_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")


Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


In [8]:
# Save PM2.5 Data Frame Sensors into CSV file
df_to_csv(df_pm25_raw, "pm25_raw.csv")